# Projeto

**Objetivo**: Desenvolver um CRUD utilizando um arquivo JSON para "persistir" os dados e aplicar algumas outras funcionalidades que seriam interessantes no sistema.

**Requisitos**: O projeto deve conter as seguintes funcionalidades/ características:

- Utilizar JSON para persistir os dados
- Função de criar o arquivo JSON
- Função para adicionar registros (através do *input*, ou de um conjunto de dados históricos, por exemplo)
- Função para realizar o *udpate* (atualização) das informações
- Função para excluir registros
- Função para realizar a leitura de registros
- Função para filtrar informações dos dados
- Função para aplicar alguma regra de negócio (varia de acordo com o sistema, exemplo, grupo criou um sistema de consulta de colaboradores, logo deseja-se ter uma função que calcula o faturamento de todos os funcionários que são vendedores)
- Função que retorne alguma estatística dos dados
- Exemplos de funcionamento de cada uma das funções acima

**Pontos importantes**:

- Organização de código no ``jupyter notebook`` (não precisa de ppt, no dia apenas o notebook será necessário)
- Definição sobre o que será o sistema (livre e definido pelo grupo)
- Apresentação do sistema e funcionalidades (assim como os exemplos). Não precisa ser o grupo todo apresentando
- Gerenciamento de erros, como no caso de num input o usuário colocar 'abc' num campo de valor inteiro
- A nota do grupo será a nota de cada integrante
- Tempo de apresentação: de 15 a 20 minutos
- A princípio todos integrantes devem entregar o mesmo notebook no sistema do LMS, para fins de retorno de nota
- Não precisa se ater somente aos 10 requisitos acima, pode ir além :)

banco de dados
https://www.kaggle.com/datasets/iamsouravbanerjee/customer-shopping-trends-dataset

In [4]:
# leitura de registros
import csv
lista_vendas = list()
with open("shopping_trends.csv", "r") as arquivo:
    arquivo_csv = csv.reader(arquivo, delimiter=",")
    next(arquivo_csv)  # pula a primeira linha (cabeçalho)
    for linha in enumerate(arquivo_csv):
        #print(linha)
        Customer_ID, Age, Gender, Item_Purchased,Category, Purchase_Amount, Location, Size, Color, Season, Review_Rating, Subscription_Status, Payment_Method, Shipping_Type, Discount_Applied, Promo_Code_Used, Previous_Purchases, Preferred_Payment_Method, Frequency_of_Purchases = linha[1]
        #print('ID', Customer_ID,'Idade', Age,'Genero', Gender,'Item_Comprado',Item_Purchased,
        #                          'Categoria', Category,'Metodo_Pagamento', Payment_Method)
        lista_vendas.append({'ID': Customer_ID, #numérica
                            'Idade': Age, #numérica
                            'Genero': Gender, #categórica
                            'Item_Comprado':Item_Purchased, #categórica
                            'Categoria': Category, #categórica
                            'Metodo_Pagamento': Payment_Method, #categórica
                            'Quantidade_Compras_Anteriores': Previous_Purchases, #numérica
                            'Frequencia_Compra': Frequency_of_Purchases #categórica
                            })


In [30]:
# algum tipo de estatística

soma_Male   = sum([ 1 for i in lista_vendas if i['Genero'] == 'Male'  ])
soma_Female = sum([ 1 for i in lista_vendas if i['Genero'] == 'Female'])
total = len(lista_vendas)
porcentagem_F = soma_Female/total*100
porcentagem_M = soma_Male/total*100
print(f'{porcentagem_F}% das compras foram realizadas por mulheres e {porcentagem_M}% da compras realizadas por homens.')

32.0% das compras foram realizadas por mulheres e 68.0% da compras realizadas por homens


In [ ]:
# algum tipo de regra de negócio
# pesquisar o gênero e a faixa de idade que mais consome
# pesquisar a categoria ou produto mais ou menos consumida


In [8]:
# algum tipo de filtro
filtro = [ i for i in lista_vendas if i['Genero'] == 'Male'  ]
filtro

[{'ID': '1',
  'Idade': '55',
  'Genero': 'Male',
  'Item_Comprado': 'Blouse',
  'Categoria': 'Clothing',
  'Metodo_Pagamento': 'Credit Card'},
 {'ID': '2',
  'Idade': '19',
  'Genero': 'Male',
  'Item_Comprado': 'Sweater',
  'Categoria': 'Clothing',
  'Metodo_Pagamento': 'Bank Transfer'},
 {'ID': '3',
  'Idade': '50',
  'Genero': 'Male',
  'Item_Comprado': 'Jeans',
  'Categoria': 'Clothing',
  'Metodo_Pagamento': 'Cash'},
 {'ID': '4',
  'Idade': '21',
  'Genero': 'Male',
  'Item_Comprado': 'Sandals',
  'Categoria': 'Footwear',
  'Metodo_Pagamento': 'PayPal'},
 {'ID': '5',
  'Idade': '45',
  'Genero': 'Male',
  'Item_Comprado': 'Blouse',
  'Categoria': 'Clothing',
  'Metodo_Pagamento': 'Cash'},
 {'ID': '6',
  'Idade': '46',
  'Genero': 'Male',
  'Item_Comprado': 'Sneakers',
  'Categoria': 'Footwear',
  'Metodo_Pagamento': 'Venmo'},
 {'ID': '7',
  'Idade': '63',
  'Genero': 'Male',
  'Item_Comprado': 'Shirt',
  'Categoria': 'Clothing',
  'Metodo_Pagamento': 'Debit Card'},
 {'ID': '8',
 

In [23]:
# dict para json
import json
json_vendas = json.dumps(lista_vendas, indent=2)

print(json_vendas)

[
  {
    "ID": "1",
    "Idade": "55",
    "Genero": "Male",
    "Item_Comprado": "Blouse",
    "Categoria": "Clothing",
    "Metodo_Pagamento": "Credit Card",
    "Quantidade_Compras_Anteriores": "14",
    "Frequencia_Compra": "Fortnightly"
  },
  {
    "ID": "2",
    "Idade": "19",
    "Genero": "Male",
    "Item_Comprado": "Sweater",
    "Categoria": "Clothing",
    "Metodo_Pagamento": "Bank Transfer",
    "Quantidade_Compras_Anteriores": "2",
    "Frequencia_Compra": "Fortnightly"
  },
  {
    "ID": "3",
    "Idade": "50",
    "Genero": "Male",
    "Item_Comprado": "Jeans",
    "Categoria": "Clothing",
    "Metodo_Pagamento": "Cash",
    "Quantidade_Compras_Anteriores": "23",
    "Frequencia_Compra": "Weekly"
  },
  {
    "ID": "4",
    "Idade": "21",
    "Genero": "Male",
    "Item_Comprado": "Sandals",
    "Categoria": "Footwear",
    "Metodo_Pagamento": "PayPal",
    "Quantidade_Compras_Anteriores": "49",
    "Frequencia_Compra": "Weekly"
  },
  {
    "ID": "5",
    "Idade": "4

In [10]:
# salvando json em arquivo
arquivo = open('dados_vendas.txt', 'w') # cria um arquivo ola.txt
arquivo.write(json_vendas) # escreve "Olá mundo" no arquivo
arquivo.close() # fecha e salva o arquivo